# Linear Algebra

It is safe to say that linear algebra is the cornerstone of modern mathematical physics, providing the essential tools for understanding and solving a wide range of problems. From the study of vector spaces and transformations to eigenvalue problems and matrix operations, linear algebra forms the foundation for many physical theories. Whether in classical mechanics, quantum physics, relativity, or electrodynamics, linear algebra enables the representation of physical quantities, the formulation of physical laws, and the manipulation of complex systems in an efficient and structured way. Its concepts permeate the formulation of everything from equations of motion to the quantum states of particles, making it indispensable for both theoretical insights and practical computations.

Opposed to the widespread disbelief, vectors are not merely 1D arrays, and tensors are not simply multi-dimensional arrays either. While it’s common to represent vectors and tensors in this form for computational convenience, their true nature is far more profound. Vectors exist in an abstract vector space and only take on specific numerical values when projected onto a basis. Similarly, tensors are generalized multi-linear mappings that relate vectors, scalars, and other tensors in ways that transcend their array-like representations. Their intrinsic properties remain invariant under transformations, making them essential tools in describing physical phenomena, from forces and fields to stress and strain in materials, in a way that doesn’t depend on arbitrary choices of coordinates.

The linear algebra submodule in `sigmaepsilon.math` provides **implementations of Vectors and Tensors as they are fundamentally understood in physics.**

## Arrays

**Arrays** are ordered collections of elements, typically numbers, arranged in one or more dimensions. In mathematics and computing, they represent data in a structured form, making it easier to manipulate large datasets. A **one-dimensional array** is a simple list (similar to a vector), while **multi-dimensional arrays** can represent more complex structures like matrices or tensors (but they are not tensors). Arrays are fundamental in fields like data science, machine learning, and numerical computation, where they are used to store and process large volumes of data efficiently.

### Array

### JaggedArray

In [1]:
from sigmaepsilon.math.linalg import JaggedArray
import numpy as np

data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
arr = JaggedArray(data, cuts=[3, 3, 3])
arr

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [2]:
type(arr._wrapped)

numpy.ndarray

In [3]:
arr.is_jagged()

False

In [4]:
arr.widths()

array([3, 3, 3], dtype=int64)

The underlying array in this case is a NumPy array and it should behave like one, you can use it as a drop-in replacement. However, the constructor of the class accepts a `force_numpy` parameter, which is `True` by default. It means that the constructor will try to use NumPy whenever possible and only uses Awkward if the input is truly jagged.

In [5]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
arr = JaggedArray(data, cuts=[3, 3, 4])
arr

<Array [[1, 2, 3], [4, 5, 6], [7, 8, 9, 10]] type='3 * var * int32'>

In [6]:
type(arr._wrapped)

awkward.highlevel.Array

In [7]:
arr.is_jagged()

True

In [8]:
arr.widths()

array([3, 3, 4], dtype=int64)

The class generalizes some NumPy functions to jagged arrays, like `unique` and `concatenate`.

In [9]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 9])
arr = JaggedArray(data, cuts=[3, 3, 4])
np.unique(arr)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
arr1 = JaggedArray(data, cuts=[3, 3, 4])

data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
arr2 = JaggedArray(data, cuts=[3, 2, 6])

np.concatenate([arr1, arr2])

<Array [[1, 2, 3], [4, ...], ..., [6, 7, 8, 9, 10, 11]] type='6 * var * int32'>

In [11]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 9])
arr = JaggedArray(data, cuts=[3, 3, 4])
arr = JaggedArray(arr.to_array()*3)
arr

<Array [[3, 6, 9], [12, ..., 18], [21, 24, 27, 27]] type='3 * var * int32'>

In [12]:
arr.to_array()  # returns a NumPy or an Awkward array
arr.to_ak()     # returns an Awkward array
arr.to_numpy()  # returns a dense NumPy array
arr.to_list()   # returns a list of lists
arr.to_csr()    # returns an instance of sigmaepsilon.math.linalg.sparse.csr_matrix
arr.to_scipy()  # returns an instance of scipy.sparse.csr_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (3, 4)>

### csr_matrix

In [13]:
from scipy.sparse import csr_matrix as csr_scipy
from sigmaepsilon.math.linalg import csr_matrix

# create a JaggedArray and convert it to a scipy.sparse.csr_matrix
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
scipy_matrix = JaggedArray(data, cuts=[3, 3, 4]).to_csr()

# create a csr_matrix instance from a dense NumPy array
scipy_matrix = csr_scipy((3, 4), dtype=np.int8).toarray()
csr_matrix(scipy_matrix)

3x4 CSR matrix of 12 values.

In [14]:
from numba import jit


@jit(nopython=True, nogil=True)
def numba_nopython(csr: csr_matrix, i: int):
    csr.data
    csr.indices
    csr.indptr
    csr.shape
    return csr.row(i), csr.irow(i)


row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
matrix = csr_scipy((data, (row, col)), shape=(3, 3))

csr = csr_matrix(matrix)
numba_nopython(csr, 0)

(array([1., 2.]), array([0, 2]))

## Reference Frames

In [15]:
from sigmaepsilon.math.linalg import ReferenceFrame, RectangularFrame, CartesianFrame

Let's say you get your hands on a cartesian frame (one with orthonormal basis vectors).

In [16]:
frame = CartesianFrame(dim=3)

Now there are some operations you can apply on a frame. Some of these operations would result in another frame, which is not cartesian. Some operations would modify the frame inplace and result in the instance losing the property of orthogonality or normality. For instance, if all basis vectors were multiplied by a scalar, the new basis vectors would be still orthogonal, but not normal.

In [17]:
type(frame*2)

sigmaepsilon.math.linalg.frame.RectangularFrame

In [18]:
type(frame+2)

sigmaepsilon.math.linalg.frame.ReferenceFrame

 The good new is that you don't really have to worry about these things as the library will choose the most appropriate class for the output, depending on the nature of the operation.

In [19]:
type(frame)

sigmaepsilon.math.linalg.frame.CartesianFrame

In [20]:
frame *=2
type(frame)

sigmaepsilon.math.linalg.frame.RectangularFrame

In [21]:
frame +=2
type(frame)

sigmaepsilon.math.linalg.frame.ReferenceFrame

In [22]:
frame.axes

Array([[4., 2., 2.],
       [2., 4., 2.],
       [2., 2., 4.]])

You can see that the basis vectors of the frame are no longer orthogonal or normed, but they are still independent.

In [23]:
frame.is_cartesian

False

In [24]:
frame.is_rectangular

False

In [25]:
frame.is_independent

True

In [26]:
frame.dual()

Array([[ 0.375, -0.125, -0.125],
       [-0.125,  0.375, -0.125],
       [-0.125, -0.125,  0.375]])

## Vectors and Tensors

**Vectors** and **tensors** are fundamental concepts in mathematics and physics, representing quantities that can describe physical phenomena. A **vector** is a one-dimensional object characterized by both magnitude and direction, commonly used to represent physical quantities like velocity, force, and displacement. A **tensor** is a more generalized mathematical object that can describe relationships between vectors or other tensors and extends across multiple dimensions. Tensors are crucial in fields like continuum mechanics, electromagnetism, and general relativity, where they model complex interactions in various coordinate systems.

### Vectors

As a basic example, consider two orthonormal frames in Euclidean space. The first one is the standard frame defined with the unit matrix $\mathbf{I}$, the second is obtained by rotating $\mathbf{I}$ around the $z$ axis with $90$ degrees. A vector is defined in $\mathbf{I}$ (the source) and we want to know it's components in the rotated frame (the target).

We begin by creating the coordinates of the vector as an 1d array.

In [27]:
import numpy as np

# the coordinates of a vector in the source frame
arr_source = np.array([3 ** 0.5 / 2, 0.5, 0])
arr_source

array([0.8660254, 0.5      , 0.       ])

In [28]:
from sigmaepsilon.math.linalg import ReferenceFrame

source_frame = ReferenceFrame(dim=3)  # this is a 3 by 3 identity matrix
target_frame = source_frame.orient_new('Body', [0, 0, 90*np.pi/180],  'XYZ')

To transform the vector into the target frame, we can use the Vector object directly, and it handles everything in the background.

In [29]:
from sigmaepsilon.math.linalg import Vector

arr_target = Vector(arr_source, frame=source_frame).show(target_frame)
arr_target

array([ 0.5      , -0.8660254,  0.       ])

### Tensors

Create a Tensor of order 6 in a frame with random components:

In [30]:
from sigmaepsilon.math.linalg import Tensor, ReferenceFrame
from numpy.random import rand

frame = ReferenceFrame(dim=3)
array = rand(3, 3, 3, 3, 3, 3)
A = Tensor(array, frame=frame)

Get the tensor in the dual frame:

In [31]:
A_dual = A.dual()

Create an other tensor, in this case a 5th-order one, and calculate their generalized dot product, which is a 9th-order tensor:

In [32]:
from sigmaepsilon.math.linalg import dot

array = rand(3, 3, 3, 3, 3)
B = Tensor(array, frame=frame)
C = dot(A, B, axes=[0, 0])
assert C.rank == (A.rank + B.rank - 2)

The library also provides dedicated tensor classes for the most common quantities of engineering practice.

### Objectivity of Tensorial Quantities

Tensors are mathematical objects that generalize scalars, vectors, and higher-order quantities, and they are defined by how they transform between different coordinate systems. This property makes tensors essential in physics, engineering, and mathematics, where the description of physical quantities must remain consistent regardless of the chosen frame of reference. A remarkable feature of the library is that tensorial quantities keep their objectivity with respect to changes of their host reference frame. To show this, create a new vector.

In [33]:
arr = np.array([1, 0, 0])
frame = ReferenceFrame(dim=3)
vector = Vector(arr, frame=frame)

Now transform the hosting frame of the vector inplace.

In [34]:
frame.orient('Body', [0, 0, -90*np.pi/180],  'XYZ')
frame.axes

Array([[ 6.123234e-17, -1.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  6.123234e-17,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00]])

If you now check the coordinates of the vector, you will see that they have changed, due to the changes in the hosting frame.

In [35]:
vector.array

Array([6.123234e-17, 1.000000e+00, 0.000000e+00])

However, the vector stayed the same. You can check this, by asking for the coordinates of the vector in the ambient frame (which in this case is the frame it was defined in).

In [36]:
vector.show()

array([1., 0., 0.])

The point is that if a quantity is tensorial, it doesn't matter which frame you define the quantity in, since its meaning is independent from the hosting frame. Therefore, if the hosting frame changes, the coordinates of the tensorial quantity must change as well. A more trivial example is the following:

In [37]:
arr = np.array([1, 0, 0])
frame = ReferenceFrame(dim=3)
vector = Vector(arr, frame=frame)
frame *= 2
vector.array

Array([0.5, 0. , 0. ])

It makes perfect sense. The length of the basis vectors doubled, so the coordinates shrunk to half of the original values. Resetting the basis vectors (axes) of the hosting frame, resets the coordinates of the vector as well.

In [38]:
frame.axes = np.eye(3)
vector.array

Array([1., 0., 0.])

In [39]:
from sigmaepsilon.math.linalg import random_posdef_matrix

random_axes = random_posdef_matrix(3) * 100
frame.axes = random_axes
vector.show()

array([ 1.00000000e+00, -2.25984087e-14, -2.22766958e-14])

In [41]:
frame.is_rectangular, frame.is_cartesian

(False, False)

It is clear that the coordinates of the vector are still the same in the ambient frame as they were at the beginning and the next cell shows that if we take the dot product of the vector with itself, the result is still $1.0$, even though the coordinates of it have changed and the frame of it is a random one.

In [40]:
from sigmaepsilon.math.linalg import dot

dot(vector, vector)

0.9999999999999374

Again, this demonstrates that if a quantity is tensorial, the specific frame in which it is represented as an array is less important than ensuring it has a frame. This flexibility is valuable because, in many practical problems, it's easier to define a tensor's components in one particular frame. By having the freedom to choose the most advantageous frame, we can simplify the process of solving the problem while maintaining the tensor's properties. The next example shows that if we define two vectors, and we repeatedly alter their frames, their dot product doesn't change.

In [53]:
arr = np.array([1, 0, 0])
frameA = ReferenceFrame(dim=3)
vectorA = Vector(arr, frame=frameA)
frameB = ReferenceFrame(dim=3)
vectorB = Vector(arr, frame=frameB)

print("Coordinates of vector A: ", vectorA.array)
print("Coordinates of vector B: ", vectorB.array)
print("dot(vectorA, vectorB): ", dot(vectorA, vectorB))
print("dot(arrayA, arrayB): ", dot(vectorA.array, vectorB.array))
print(63*"-")

for _ in range(3):
    random_axes = random_posdef_matrix(3) * 100
    frameA.axes = random_axes
    random_axes = random_posdef_matrix(3) * 100
    frameB.axes = random_axes
    print("Coordinates of vector A: ", vectorA.array)
    print("Coordinates of vector B: ", vectorB.array)
    print("dot(vectorA, vectorB): ", dot(vectorA, vectorB))
    print("dot(arrayA, arrayB): ", dot(vectorA.array, vectorB.array))

Coordinates of vector A:  [1. 0. 0.]
Coordinates of vector B:  [1. 0. 0.]
dot(vectorA, vectorB):  1.0
dot(arrayA, arrayB):  1.0
---------------------------------------------------------------
Coordinates of vector A:  [ 0.14003901  0.11117086 -0.2200233 ]
Coordinates of vector B:  [ 0.04139533 -0.00550249 -0.03643786]
dot(vectorA, vectorB):  1.000000000000006
dot(arrayA, arrayB):  0.013202422509482573
Coordinates of vector A:  [ 4.00354869 -4.19724454 -0.14587514]
Coordinates of vector B:  [ 2.64746757  0.48450066 -1.39766909]
dot(vectorA, vectorB):  1.0000000000021052
dot(arrayA, arrayB):  8.769582746424097
Coordinates of vector A:  [ 2.06055094 -1.64386404 -1.55690927]
Coordinates of vector B:  [ 3.55832797  0.04023036 -3.09062927]
dot(vectorA, vectorB):  0.9999999999802368
dot(arrayA, arrayB):  12.077812168548878


This explains why it is incorrect to say that vectors are 1d arrays. They are only representations of vectors in the space of some basis vectors, but they are not the vectors themselves. You can also observe how the dot product of the two vectors remains the same, but the dot product of their respective arrays changes.